In [1]:
import os
import xlutils
import databaker
import databaker.constants
from databaker.constants import *
import xypath
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values, savepreviewhtmlBAGS
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
import zipfile, io, re, os


In [3]:
from tempfile import NamedTemporaryFile

dirname = "table7zips"
dfiles = os.listdir(dirname)
dfiles.sort(reverse=True)
dfile = dfiles[0]
zfile = os.path.join(dirname, dfile)
zfile

zdir = zipfile.ZipFile(zfile)
xlsfilenames = zdir.namelist()
xlsfilename = xlsfilenames[0]

z = zdir.read(xlsfilename)
print(len(z))
z[:100]
tf = NamedTemporaryFile("w+b", suffix=".xls")
tf.write(z)
tableset = xypath.loader.table_set(tf.name, extension='xls')


893440


In [4]:
tabs = list(xypath.loader.get_sheets(tableset, "*"))
for tab in tabs:
    print(tab.name)

Notes
All
Male
Female
Full-Time
Part-Time
Male Full-Time
Male Part-Time
Female Full-Time
Female Part-Time


In [5]:
previewfile = "preview.html"
savepreviewhtmlBAGS(tabs[1], previewfile)

opening file preview.html
table 'All' written


In [6]:

def readzxls(dfile, zdir, xlsfilename):
    print("loading: '%s'" % (xlsfilename))
    from tempfile import NamedTemporaryFile
    tf = NamedTemporaryFile("w+b", suffix=".xls")
    tf.write(zdir.read(xlsfilename))
    tableset = xypath.loader.table_set(tf.name, extension='xls')
    tabs = list(xypath.loader.get_sheets(tableset, "*"))
    assert tabs[0].name in ["Notes", "CV notes"], tabs[0].name
    return tabs[1:]



In [7]:
previewfile = "preview.html"

def pertab(dfile, xlsfilename, tab):
    earningstype = re.search("   ([^\d]*) \d", xlsfilename).group(1)  # becomes PARAM(0)
    constantdimensions = [ (earningstype, "Earnings", None, None), 
                           (re.search("(Male|Female|All)", tab.name+" All").group(1), "Sex", None, None),
                           (re.search("(Full-Time|Part-Time|All)", tab.name+" All").group(1), "Working Pattern", None, None), 
                           (re.match("(\d\d\d\d)", dfile).group(1), "Year", None, None)
                         ]
    dimensions = [ 
        (tab.excel_ref('3').expand(RIGHT).is_not_blank().is_not_whitespace(), "Default Title 1", DIRECTLY, ABOVE), 
        (tab.excel_ref('4').expand(RIGHT).is_not_blank().is_not_whitespace(), "Default Title 2", DIRECTLY, ABOVE), 
        (tab.excel_ref('5').expand(RIGHT).is_not_blank().is_not_whitespace(), "Default Title 3", DIRECTLY, ABOVE), 

        (tab.excel_ref('A6').expand(DOWN), "NoBold", DIRECTLY, LEFT),
        (tab.excel_ref('B6').expand(DOWN), "N", DIRECTLY, LEFT)
                ]

    obs = tab.excel_ref('C6').expand(RIGHT).expand(DOWN)
    obs = obs - tab.filter(contains_string ('CV > 10% and <= 20%')).shift(-1, -3).expand(RIGHT).expand(DOWN)
    obs = obs - tab.excel_ref('A1').fill(DOWN).filter(contains_string("Not Classified")).expand(RIGHT).expand(DOWN)
    dimensions.extend(constantdimensions)
    return (tab, dimensions, obs)



In [ ]:
conversionsegments = [ ]

# whole directory
dirname = "table7zips"
dfiles = os.listdir(dirname)
dfiles.sort(reverse=True)

for dfile in dfiles:
    zfile = os.path.join(dirname, dfile)
    zdir = zipfile.ZipFile(zfile)
    xlsfilenames = zdir.namelist()
    print(dfile)
    for xlsfilename in xlsfilenames:
        tabs = readzxls(dfile, zdir, xlsfilename)
        for tab in tabs:
            conversionsegments.append(pertab(dfile, xlsfilename, tab))

#savepreviewhtml(conversionsegment, batchcelllookup, previewfile)


2016-table72016provisional.zip
loading: 'PROV - Work Geography Table 7.1a   Weekly pay - Gross 2016.xls'
loading: 'PROV - Work Geography Table 7.1b   Weekly pay - Gross 2016 CV.xls'


'CV notes'